The NOAA HRRR is a real-time 3km resolution, hourly updated, cloud-resolving, convection-allowing atmospheric model, initialized by 3km grids with 3km radar assimilation.

This notebook provides an example of accessing HRRR data, including (1) finding the data file corresponding to a date and time, (2) retrieving a portion of that file from blob storage which includes the surface temperature variable, (3) opening the file using the xarray library, and (4) rendering an image of the forecast.

This dataset is documented at http://aka.ms/ai4edata-hrrr.

In [5]:
import climatedata_functions as climf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np



In [ ]:
metadata = pd.read_csv("../data/metadata.csv")
ds = climf.get_ds()#only works with recent dates

### Make metadafile with gridpoints

In [ ]:
#tranform coordinates for longitude
metadata['longitude_trans'] = metadata['longitude']+360 
#add columns for gridpoints
metadata['x_grid'] = ''
metadata['y_grid'] = ''

#make new metadatafile and save as cvs 
#metadata_new = climf.save_grids(metadata.head(), ds) #(takes approx 6 minutes)
#metadata_new.to_csv('../data/metadata_grids.csv')


In [4]:
#load new metadata file that includes the gridpoints
metadata = pd.read_csv("../data/metadata_grids.csv")

In [3]:
metadata.head(3)

NameError: name 'metadata' is not defined

### Get temperatures for gridpoints

Make function that gets the mean/median temperature for one date for all the places sampled at that date (to save time) and the previos x days

In [6]:
metadata = pd.read_csv("../data/metadata_temp_12_01_06_complete_without_a.csv")

**Data list:**

**Radiation**
* DSWRF:surface	(Downward Short-Wave Radiation Flux [W/m^2])
* DLWRF:surface	(Downward Long-Wave Rad. Flux [W/m^2])

**Wind**
* WIND:10 m above ground (0-0 day max f	Wind Speed [m/s])
* UGRD:10 m above ground (analysis	U-Component of Wind [m/s]) --> also avalable as 0-0 day max)
* VGRD:10 m above ground (analysis	V-Component of Wind [m/s]) --> also avalable as 0-0 day max)

**Temperature**
* maybe think abut getting a different time (UTS vs time at a specific place...)

In [3]:
#define  download parameters
days = 7  #how many days to go back?
hour = 12 #which our of the day (UTS time!!!)
param_layer = ':DSWRF:surface' # :TMP:surface  surface temperature, #available parameters and layes: https://www.nco.ncep.noaa.gov/pmb/products/hrrr/hrrr.t00z.wrfsfcf00.grib2.shtml
forecast_param = 'dswrf'#name of the column in the metadata table
#dswrf = Downward Short-Wave Radiation Flux [W/m^2] https://www.goes-r.gov/products/baseline-DSR.html

In [ ]:
from datetime import timedelta
import warnings
warnings.simplefilter(action='ignore')

#make empty list to store the dates already sampled
done_list = []

metadata[forecast_param+'_'+str(hour)] = '' #create new column to store the data 
#getting the data
for idx,row in enumerate(metadata.index): #takes first element in metadata list
    #print(row, idx)
    start_date = metadata.date[idx]
    date_index_list = metadata.index[metadata.date == start_date]#list with all the indexes of dates with the same date
    #print(date_index_list)
    if start_date not in done_list:#if data for the given date was already downloaded, this row will be skipped
        temp_list = [[0] * days for i in range(len(date_index_list))]#make list of lists to store the values inside
        done_list.append(start_date)#list of dates already samples
        start_date = climf.get_start_date(start_date)#formate to time object
        print(start_date)
        for x in range(days):
            count = 0
            #x = x*2 (if we want to take only every second day)
            day_date = start_date - timedelta(days=x)
            #ds, stop = climf.get_ds_aws_array(day_date, hour,param_layer, forecast_param)#getting the temperature array for the specified date
            ds, stop = climf.get_ds_aws_array(day_date, hour,param_layer, forecast_param)#getting the temperature array for the specified date
            print(ds, stop)
            for index in date_index_list:
                x_grid = metadata.x_grid[index]
                y_grid = metadata.y_grid[index]
                if stop == True:
                    temp_list[count][x] = np.nan
                else:
                    temp_list[count][x] = ds[x_grid][y_grid]
                count += 1
                if x == days-1 and index == date_index_list[len(date_index_list)-1]:#if condition is met put the values in the metadata file
                    for i in range(len(temp_list)):                    
                        metadata[forecast_param+'_'+str(hour)].loc[date_index_list[i]] = temp_list[i] #not index but  
    else:
        continue 

In [8]:
metadata.to_csv('../data/metadata_temp_06_12_dswrf_12_complete.csv')

In [ ]:
from datetime import timedelta
import warnings
warnings.simplefilter(action='ignore')

#make empty list to store the dates already sampled
done_list = []

#define important download parameters
days = 7
hour = 1 #which our of the day, impoetant: also change in the columnnames 2 times!!!)
metadata['temp_01'] = '' #create new column to dstore the data #change also in last line before the else!!!

#getting the data
for idx,row in enumerate(metadata.index): #takes first element in metadata list
    #print(row, idx)
    start_date = metadata.date[idx]
    date_index_list = metadata.index[metadata.date == start_date]#list with all the indexes of dates with the same date
    #print(date_index_list)
    if start_date not in done_list:#if data for the given date was already downloaded, this row will be skipped
        temp_list = [[0] * days for i in range(len(date_index_list))]#make list of lists to store the values inside
        done_list.append(start_date)#list of dates already samples
        start_date = climf.get_start_date(start_date)#formate to time object
        print(start_date)
        for x in range(days):
            count = 0
            day_date = start_date - timedelta(days=x)
            ds, stop = climf.get_ds_aws(day_date, hour)#getting the temperature array for the specified date
            for index in date_index_list:
                x_grid = metadata.x_grid[index]
                y_grid = metadata.y_grid[index]
                if stop == True:
                    temp_list[count][x] = np.nan
                else:
                    temp_list[count][x] = ds[x_grid][y_grid]
                count += 1
                if x == days-1 and index == date_index_list[len(date_index_list)-1]:#if condition is met put the values in the metadata file
                    for i in range(len(temp_list)):                    
                        metadata.temp_01.loc[date_index_list[i]] = temp_list[i] #not index but  
    else:
        continue 

2018-05-14
2016-08-31
2020-11-19
2016-08-24
2019-07-23
2021-08-23
2017-11-15
2020-06-10
2014-08-12
2018-06-27
2013-11-06
2015-08-24
2014-11-01
2021-10-18
2015-08-26
2019-08-26
2018-01-08
2015-08-18
2017-07-28
2015-07-07
2018-06-13
2018-01-09
2013-07-09
2020-08-13
2013-08-17
2018-08-13
2017-03-09
2013-07-29
2017-12-05
2019-06-06
2020-02-26
2018-04-11
2020-05-18
2016-07-20
2018-08-27
2020-03-18
2019-01-17
2016-06-21
2015-01-13
2016-06-08
2016-03-08
2013-01-15
2021-07-14
2017-06-19
2016-04-13
2019-07-02
2019-05-22
2014-02-28
2014-07-15
2017-08-01
2019-09-24
2017-09-26
2016-01-19
2013-08-20
2017-08-02
2016-06-22
2015-06-10
2018-04-03
2016-04-06
2016-05-17
2017-10-05
2013-07-16
2016-07-19
2014-12-03
2016-08-05
2020-05-12
2015-09-25
2017-06-12
2017-03-31
2019-03-19
2017-08-28
2015-04-22
2019-08-22
2016-05-24
2016-12-13
2015-04-14
2020-06-24
2014-07-22
2020-08-11
2018-06-20
2015-11-16
2017-08-25
2020-11-10
2020-07-15
2013-09-23
2013-02-12
2015-06-22
2015-05-06
2014-06-23
2021-08-16
2016-09-20

If the download was interupted the code in the next cell can be used to continue with the download. Since the dates already dowloaded are saved in 'done_list', it can continue from that withouth starting from scratch.

In [ ]:
done_list.pop()#remove last element (in case it didn work with that)
len(done_list)#how many dates are already inside (1637 unique dates in total)

In [ ]:
#
import warnings
warnings.simplefilter(action='ignore')


temp_list = []
#done_list = [] deactivate because we want to continue from the old donelist
days = 7
hour = 12 #which our of the day


for idx,row in enumerate(metadata.index): #takes first element in metadata list
    #print(row, idx)
    start_date = metadata.date[idx]
    date_index_list = metadata.index[metadata.date == start_date]#list with all the indexes of dates with the same date
    #print(date_index_list)
    if start_date not in done_list:#only do if this date wasnt used before
        temp_list = [[0] * days for i in range(len(date_index_list))]#make list of lists to store the values inside
        #print(len(temp_list), len(temp_list[0]))
        done_list.append(start_date)#list of dates already samples
        start_date = climf.get_start_date(start_date)#formate to time object
        print(start_date)
        for x in range(days):
            count = 0
            day_date = start_date - timedelta(days=x)
            ds, stop = climf.get_ds_aws(day_date, hour)#getting the temperature array for the specified date
            for index in date_index_list:
                x_grid = metadata.x_grid[index]
                y_grid = metadata.y_grid[index]
                if stop == True:
                    temp_list[count][x] = np.nan
                else:
                    temp_list[count][x] = ds[x_grid][y_grid]
                count += 1
                if x == days-1 and index == date_index_list[len(date_index_list)-1]:#if condition is met put the values in the metadata file
                    for i in range(len(temp_list)):                    
                        metadata.temp_01.loc[date_index_list[i]] = temp_list[i] #not index but  
    else:
        continue 

In [ ]:
#metadata.to_csv('../data/metadata_temp_12_01_06_complete.csv')